# Reinforcement Learning: Theory and Algorithms

第１章のまとめ＋Python実装を行っていきます．

# マルコフ決定過程
## 割引無限マルコフ決定過程
以下のようにタプルで表現できます．
$$
M=(\mathcal{S}, \mathcal{A}, P, r, \gamma, \mu)
$$
- 状態空間：$\mathcal{S}$，有限または可算無限であると仮定
- 行動空間：$\mathcal{A}$，有限であると仮定
- 遷移関数：$P: \mathcal{S} \times \mathcal{A} \rightarrow \Delta(\mathcal{S})$
	- $P_{s, a} := P(\cdot \mid s, a)$ 
- 報酬関数：$r: \mathcal{S} \times \mathcal{A} \rightarrow[0,1]$
- 割引率：$\gamma \in [0, 1)$
- 初期状態の分布：$\mu \in \Delta(\mathcal{S})$

* 軌跡：$\tau_t=\left(s_0, a_0, r_0, s_1, \ldots, s_t, a_t, r_t\right)$
* すべての軌道集合：**$\mathcal{H}$
* 履歴依存方策：
$$
\pi: \mathcal{H} \rightarrow \Delta(\mathcal{A})
$$
* 定常方策(非履歴依存)：
	* 確率論的，確率論的：
$$
\pi: \mathcal{S} \rightarrow \Delta(\mathcal{A})
$$
$$
\pi: \mathcal{S} \rightarrow \mathcal{A}
$$

## 価値関数
* $M$，$\pi$に従う状態価値関数 $V_M^\pi: \mathcal{S} \rightarrow \mathbb{R}$： 
$$
V_M^\pi(s)=\mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t r\left(s_t, a_t\right) \mid \pi, s_0=s\right] .
$$
* $M$，$\pi$に従う行動価値関数　$Q_M^\pi: \mathcal{S} \times \mathcal{A} \rightarrow \mathbb{R}$：
$$
Q_M^\pi(s, a)=\mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t r\left(s_t, a_t\right) \mid \pi, s_0=s, a_0=a\right]
$$
**目標**：$\max _\pi V_M^\pi(s)$を満たす$\pi$を求める．
## 行列方式
*   $Q^\pi \in \mathbb{R}^{|\mathcal{S}||\mathcal{A}|}$:$(s, a)$成分は $Q^\pi(s, a)$
*   $r \in \mathbb{R}^{|\mathcal{S}||\mathcal{A}|}$:$(s, a)$成分は $r(s, a)$
*   $V^\pi \in \mathbb{R}^{|\mathcal{S}|}$:$s'$ 成分は $V^\pi(s')$
*   $P$: $(|\mathcal{S}||\mathcal{A}|) \times |\mathcal{S}|$ ，$(s, a)$ 行，$s'$ 列の要素は $P_{(s, a), s'} = P(s'|s, a)$．
*   $P^\pi$: $(|\mathcal{S}||\mathcal{A}|) \times (|\mathcal{S}||\mathcal{A}|)$， $(s, a)$ 行，$(s', a')$ 列の要素は $P_{(s, a), (s', a')}^\pi = P(s'|s, a) \pi(a'|s')$．

## 定常方策のベルマン方程式

### **補題1.4**：ベルマンConsistency方程式
$\pi$が定常方策とすると，$V^\pi$と$Q^\pi$はすべての $s \in \mathcal{S}$, $a \in \mathcal{A}$ に対して，以下の方程式を満たします．
$$
\begin{aligned}
V^\pi(s) & =Q^\pi(s, \pi(s)) . \\
Q^\pi(s, a) & =r(s, a)+\gamma \mathbb{E}_{a \sim \pi(\cdot \mid s), s^{\prime} \sim P(\cdot \mid s, a)}\left[V^\pi\left(s^{\prime}\right)\right] .
\end{aligned}
$$
### **証明**
****
#### **$V^\pi(s) = \mathbb{E}_{a \sim \pi(\cdot | s)}[Q^\pi(s, a)]=Q^\pi(s, \pi(s))$ の証明**

 $V^\pi(s)$ の定義を振り返りましょう．
$$
V^\pi(s) = \mathbb{E}\left[ \sum_{t=0}^\infty \gamma^t r(s_t, a_t) \mid \pi, s_0 = s \right]
$$
期待値の法則を適用し， $a_0 \sim \pi(\cdot|s_0)$ (ここで $s_0=s$)で条件付けを行います．
$$
V^\pi(s) = \mathbb{E}_{a_0 \sim \pi(\cdot | s)} \left[ \mathbb{E}\left[ \sum_{t=0}^\infty \gamma^t r(s_t, a_t) \mid \pi, s_0 = s, A_0=a_0 \right] \mid \pi, s_0=s \right]
$$
 $\mathbb{E}\left[ \sum_{t=0}^\infty \gamma^t r(s_t, a_t) \mid \pi, s_0 = s, A_0=a_0 \right]$ は， $Q^\pi(s, a_0)$ の定義そのものです．したがって，
$$
V^\pi(s) = \mathbb{E}_{a_0 \sim \pi(\cdot | s)} \left[ Q^\pi(s, a_0) \right]
$$
変数名を $a_0$ から $a$ に変更すると，補題の結果が得られます．
$$
V^\pi(s) = \mathbb{E}_{a \sim \pi(\cdot | s)} \left[ Q^\pi(s, a) \right]
$$
****
#### **$Q^\pi(s, a) = r(s, a) + \gamma \mathbb{E}_{s' \sim P(\cdot | s, a)}[V^\pi(s')]=r(s, a)+\gamma \mathbb{E}_{a \sim \pi(\cdot \mid s), s^{\prime} \sim P(\cdot \mid s, a)}\left[V^\pi\left(s^{\prime}\right)\right]$ の証明**

$Q^\pi(s, a)$ の定義を振り返りましょう．
$$
Q^\pi(s, a) = \mathbb{E}\left[ \sum_{t=0}^\infty \gamma^t r(s_t, a_t) \mid \pi, s_0 = s, a_0 = a \right]
$$
$t=0$を分離します．
$$
Q^\pi(s, a) = \mathbb{E}\left[ r(s_0, a_0) + \sum_{t=1}^\infty \gamma^t r(s_t, a_t) \mid \pi, s_0 = s, a_0 = a \right]
$$
期待値の線形性を使います．
$$
Q^\pi(s, a) = \mathbb{E}\left[ r(s_0, a_0) \mid \pi, s_0 = s, a_0 = a \right] + \mathbb{E}\left[ \sum_{t=1}^\infty \gamma^t r(s_t, a_t) \mid \pi, s_0 = s, a_0 = a \right]
$$
第1項は，方策に依存した確率変数がないので，期待値を外します．
$$
Q^\pi(s, a) = r(s, a) + \mathbb{E}\left[ \sum_{t=1}^\infty \gamma^t r(s_t, a_t) \mid \pi, s_0 = s, a_0 = a \right]
$$
第2項について，$\gamma$ をくくり出し，時間インデックスを $k = t-1$ (すなわち $t=k+1$) と変更します．
$$
\mathbb{E}\left[ \sum_{t=1}^\infty \gamma^t r(s_t, a_t) \mid \pi, s_0 = s, a_0 = a \right] = \mathbb{E}\left[ \gamma \sum_{k=0}^\infty \gamma^k r(s_{k+1}, a_{k+1}) \mid \pi, s_0 = s, a_0 = a \right]
$$
期待値の法則を適用します． $s_1 \sim P(\cdot|s, a)$で条件付けを行います．
$$
\begin{aligned}
& \gamma \mathbb{E}_{s_1 \sim P(\cdot|s, a)} \left[ \mathbb{E}\left[ \sum_{k=0}^\infty \gamma^k r(s_{k+1}, a_{k+1}) \mid \pi, s_0 = s, a_0 = a, S_1=s_1 \right] \mid \pi, s_0=s, a_0=a \right] \\
= & \gamma \mathbb{E}_{s_1 \sim P(\cdot|s, a)} \left[ \mathbb{E}\left[ \sum_{k=0}^\infty \gamma^k r(s_{k+1}, a_{k+1}) \mid \pi, S_1=s_1 \right] \right] \quad (\text{マルコフ性と定常性より})
\end{aligned}
$$
 $\mathbb{E}\left[ \sum_{k=0}^\infty \gamma^k r(s_{k+1}, a_{k+1}) \mid \pi, S_1=s_1 \right]$ は，$V^\pi(s_1)$ の定義そのものです．
$$
\mathbb{E}\left[ \sum_{k=0}^\infty \gamma^k r(s_{k+1}, a_{k+1}) \mid \pi, S_1=s_1 \right] = V^\pi(s_1)
$$
したがって，第2項は以下のように書き換えられます．
$$
\gamma \mathbb{E}_{s_1 \sim P(\cdot | s, a)} \left[ V^\pi(s_1) \right]
$$
変数名を $s_1$ から $s'$ に変更すると，
$$
\gamma \mathbb{E}_{s' \sim P(\cdot | s, a)} \left[ V^\pi(s') \right]
$$
これを元の式に代入すると，補題の結果が得られます．
$$
Q^\pi(s, a) = r(s, a) + \gamma \mathbb{E}_{s' \sim P(\cdot | s, a)} \left[ V^\pi(s') \right]
$$
****

**系1.5**
$$
Q^\pi=\left(I-\gamma P^\pi\right)^{-1} r
$$

**証明**
三角不等式などを使って導出します．そんなに難しくないです．
$$
\begin{aligned}
\left\|\left(I-\gamma P^\pi\right) x\right\|_{\infty} & =\left\|x-\gamma P^\pi x\right\|_{\infty} \\
& \geq\|x\|_{\infty}-\gamma\left\|P^\pi x\right\|_{\infty} \\
& \geq\|x\|_{\infty}-\gamma\|x\|_{\infty} \\
& =(1-\gamma)\|x\|_{\infty}>0
\end{aligned}
$$


### 定理1.7：最適ベルマン方程式

$\Pi$ をすべての定常でない、確率論的方策の集合とします．以下を定義します．
$$
\begin{aligned}
V^{\star}(s) & :=\sup _{\pi \in \Pi} V^\pi(s) \\
Q^{\star}(s, a) & :=\sup _{\pi \in \Pi} Q^\pi(s, a) .
\end{aligned}
$$
これらの値は割引無限MDPの場合，有限です．

さらに，すべての $s \in \mathcal{S}$ と $a \in \mathcal{A}$ に対して、
$$
\begin{aligned}
V^\pi(s) & =V^{\star}(s) \\
Q^\pi(s, a) & =Q^{\star}(s, a) .
\end{aligned}
$$

となる、定常かつ決定論的な方策：**最適方策** $\pi$ が存在します．

### 証明
結構トリッキーなことをします．

**TODO**






## 実装編
### ベクトル形式でベルマン方程式を書こう

$$Q^\pi=r+\gamma P V^\pi$$


In [46]:
import numpy as np 

S = 3
A = 2


r = np.zeros(S*A)
P = np.zeros((S*A, S))
gamma = 0.9
pi = np.zeros(S*A)
V_pi = np.zeros(S)


r = np.random.rand(S*A)
P = np.random.rand(S*A, S)
P = P / P.sum(axis=1)[:, None]
pi = np.random.rand(S*A)

#ベルマン方程式

Q_pi = r + gamma * np.dot(P, V_pi)

print(Q_pi)

[0.69882194 0.05180708 0.91843705 0.53129233 0.36030357 0.58303794]


$$
\begin{aligned}
Q^\pi=r+\gamma P^\pi Q^\pi .
\end{aligned}
$$

$$
P_{(s, a),\left(s^{\prime}, a^{\prime}\right)}^\pi:=P\left(s^{\prime} \mid s, a\right) \pi\left(a^{\prime} \mid s^{\prime}\right) .
$$
$P^\pi$は($S\times A$,$S\times A$)の行列です。

In [48]:
Q_pi = np.zeros(S*A)

P_pi = np.zeros((S*A,S*A))
for s in range(S):
    for a in range(A):
        for s_prime in range(S):
            for a_prime in range(A):
                P_pi[s*A+a, s_prime*A+a_prime] = P[s*A + a, s_prime] * pi[s_prime*A + a_prime]

for _ in range(100):  
    Q_pi = r + gamma * np.dot(P_pi, Q_pi)

print(Q_pi)
print(P_pi.shape)
print(np.dot(P_pi, Q_pi).shape)

[3.92621042e+11 2.87654080e+11 3.81218096e+11 3.48802854e+11
 2.48957676e+11 3.20759749e+11]
(6, 6)
(6,)



**価値関数・行動価値関数関連**

* **$Q^\pi$の解析解 (Analytical Solution for Q):** $Q^\pi=\left(I-\gamma P^\pi\right)^{-1} r$
* 最適価値 (Optimal Value):
    * $V^{\star}(s)  :=\sup _{\pi \in \Pi} V^\pi(s)$
    * $Q^{\star}(s, a)  :=\sup _{\pi \in \Pi} Q^\pi(s, a)$
* グリーディ方策 (Greedy Policy): $\pi_Q(s):=\operatorname{argmax}_{a \in \mathcal{A}} Q(s, a)$
* **ベルマン最適作用素 (Bellman Optimality Operator):** $\mathcal{T} Q:=r+\gamma P V_Q$
* アドバンテージ関数 (Advantage Function): $A^\pi(s, a):=Q^\pi(s, a)-V^\pi(s)$
* 状態訪問分布 (State Visitation Distribution): $d_{s_0}^\pi(s)=(1-\gamma) \sum_{t=0}^{\infty} \gamma^t \operatorname{Pr}^\pi\left(s_t=s \mid s_0\right)$ 
* $V_Q(s):=\max _{a \in \mathcal{A}} Q(s, a)$

**動的計画法**

* **価値反復法 (Value Iteration):** $Q \leftarrow \mathcal{T} Q$
* **方策反復法 (Policy Iteration):**
    * 1. **方策評価 (Policy Evaluation):**  $Q^{\pi_k}$ を計算
    * 2. **方策改善 (Policy Improvement):** $\pi_{k+1}=\pi_{Q^{\pi_k}}$ 


### 解析解を実装してみましょう



In [43]:
def compute_analytical_Q_pi():
    return np.linalg.inv(np.eye(S*A)-gamma * P_pi) @ r

Q_pi = compute_analytical_Q_pi()
print(Q_pi)
print(np.linalg.inv(np.eye(S*A)-gamma * P_pi).shape)
print(r.shape)

[-18.86074403 -15.98676975 -13.79090972  -9.82614407]
(4, 4)
(4,)


### ベルマン最適作用素を実装してみましょう

In [54]:
def V_Q(Q, s, A):
    start_index = s * A
    end_index = start_index + A
    return np.max(Q[start_index:end_index])

def bellman_optimality_operator():
    V = np.array([V_Q(Q_pi, s, A) for s in range(S)]) 
    return r + gamma * np.dot(P, V)



[3.23548049e+11 3.33957445e+11 3.33338632e+11 3.16347306e+11
 3.20109464e+11 3.08516471e+11]
